In [22]:
import base64
import os
import numpy as np
import time
from PIL import Image


class LZW:
    def __init__(self):
        self.imageByteArray = None
            
    def encode(self, imageFile):
        # bộ đếm kiểm tra thời gian thực thi
        start_time = time.time()
        self.convertImageToBase64(imageFile)
        filename, fileExtension = os.path.splitext(imageFile)

        result = []

        inputText = self.imageByteArray
        dictSize = 256
        dictionary = {chr(i):i for i in range(dictSize)}

        s = ""
        for c in inputText:
            temp = s + c
            if temp in dictionary:
                s = temp
            else:
                result.append(dictionary[s])
                dictionary[temp] = dictSize
                dictSize += 1
                s = c
        if s:
            result.append(dictionary[s])

        newArrayNumpy = np.array(result)

        fileSaveDic = filename + ".npy"
        np.save(fileSaveDic, newArrayNumpy)


        sizeFileInput = os.path.getsize(imageFile)
        sizeFileOutput = os.path.getsize(fileSaveDic)
        performance = ((sizeFileInput/sizeFileOutput) - 1)*100

        elapsed_time = time.time() - start_time
        print("Stat: ")

        print('- Thời gian thực thi nén : ', elapsed_time)

        new_size = os.stat(fileSaveDic).st_size
        print('- Size của file sau khi nén: ', new_size)
        
        log = "- Input File: " + imageFile + "\n- Output File: " + fileSaveDic + "\n- Preformance: " + str(performance)

        
        return  fileSaveDic, log

    def decode(self, inputPath, typeFile):
        start_time = time.time()
        filename, fileExtension = os.path.splitext(inputPath)
        outputFilename = "[Decompressed] " + filename + typeFile

        inputCode = list(np.load(inputPath))
        
        dictSize = 256
        dictionary = {i: chr(i) for i in range(dictSize)}
        result = chr(inputCode.pop(0))
        s = result
        
        for k in inputCode:
            if k in dictionary:
                entry = dictionary[k]
            elif k == dictSize:
                entry = s + s[0]
            else:
                raise ValueError('Bad compressed k: %s' % k)
            result += entry

            dictionary[dictSize] = s + entry[0]
            dictSize += 1

            s = entry
        temp = result.encode()
        dataToWriteFile = base64.b64decode(temp)

        with open(outputFilename, "wb") as f:
            f.write(dataToWriteFile)
        elapsed_time = time.time() - start_time
        print("Stat: ")
        print('- Thời gian thực thi giải nén : ', elapsed_time)
        log =  "- Input File: " + inputPath + "\n- Output File: " + outputFilename
        new_size = os.stat(outputFilename).st_size
        print('- Size của file sau khi giải nén: ', new_size)
        return outputFilename, log

    #https://stackoverflow.com/questions/3715493/encoding-an-image-file-with-base64
    def convertImageToBase64(self, image): #Lưu tối ưu
        with open(image, "rb") as f:
            data = f.read()
            stringImage = base64.b64encode(data)
        self.imageByteArray = str(stringImage)[2:]

if __name__ == "__main__":
    print('\n ... Tiến hành thuật toán nén LWZ ... \n')
    
    lzw = LZW()
    

    # sửa tuỳ vào file và định dạng như 'minh_lung_linh.png' hay 'dpt.tif' | nhớ để file cùng folder đặt file code.
    im = "RAY.BMP"

    filepath = os.path.join(os.getcwd(), im) # getcwd() là lấy cái chuỗi đang work và mình join với tên file

    oldsize = os.stat(filepath).st_size # size ở đây là kích cỡ của file from 0 -> n KB


    print(' Tên ảnh   : ', im )
    print(' Kích cỡ file  : ', oldsize ,'\n')

    encode = "RAY.npy"
    

    # nén

    print('---Thực hiện nén---')
    re, log = lzw.encode(im)
    print(log + '\n')


    # giải nén
    print('---Thực hiện giải nén---')
    re, log = lzw.decode(encode, ".BMP")
    print(log + '\n')



 ... Tiến hành thuật toán nén LWZ ... 

 Tên ảnh   :  RAY.BMP
 Kích cỡ file  :  1440054 

---Thực hiện nén---
Stat: 
- Thời gian thực thi nén :  0.8626935482025146
- Size của file sau khi nén:  768816
- Input File: RAY.BMP
- Output File: RAY.npy
- Preformance: 87.30801648248736

---Thực hiện giải nén---
Stat: 
- Thời gian thực thi giải nén :  0.5595905780792236
- Size của file sau khi giải nén:  1440054
- Input File: RAY.npy
- Output File: [Decompressed] RAY.BMP

